## Low-Rank Autoregressive Tensor Completion Imputer (LATC-TNN-imputer)

This notebook shows how to implement a LATC imputer on some real-world data sets (e.g., PeMS traffic speed data, Guangzhou traffic speed data). To overcome the problem of missing values within multivariate time series data, this method takes into account both low-rank structure and time series regression. For an in-depth discussion of LATC-imputer, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Author Name (2020). <b>Low-Rank Autorgressive Tensor Completion for Multivariate Time Series Forecasting</b>. arXiv.2006. <a href="xx" title="PDF"><b>[PDF]</b></a> 
</font>
</div>


In [1]:
import numpy as np
from numpy.linalg import inv as inv
import gc

### Define LATC-imputer kernel

We start by introducing some necessary functions that relies on `Numpy`.

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>ten2mat</code>:</b> <font color="black">Unfold tensor as matrix by specifying mode.</font></li>
<li><b><code>mat2ten</code>:</b> <font color="black">Fold matrix as tensor by specifying dimension (i.e, tensor size) and mode.</font></li>
<li><b><code>svt</code>:</b> <font color="black">Implement the process of Singular Value Thresholding (SVT).</font></li>
</ul>
</div>

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

In [3]:
def svt_tnn(mat, tau, theta):
    [m, n] = mat.shape
    if 2 * m < n:
        u, s, v = np.linalg.svd(mat @ mat.T, full_matrices = 0)
        s = np.sqrt(s)
        idx = np.sum(s > tau)
        mid = np.zeros(idx)
        mid[: theta] = 1
        mid[theta : idx] = (s[theta : idx] - tau) / s[theta : idx]
        return (u[:, : idx] @ np.diag(mid)) @ (u[:, : idx].T @ mat)
    elif m > 2 * n:
        return svt_tnn(mat.T, tau, theta).T
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    idx = np.sum(s > tau)
    vec = s[: idx].copy()
    vec[theta : idx] = s[theta : idx] - tau
    return u[:, : idx] @ np.diag(vec) @ v[: idx, :]

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [4]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

The main idea behind LATC-imputer is to approximate partially observed data with both low-rank structure and time series dynamics. The following `imputer` kernel includes some necessary inputs:

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>dense_tensor</code>:</b> <font color="black">This is an input which has the ground truth for validation. If this input is not available, you could use <code>dense_tensor = sparse_tensor.copy()</code> instead.</font></li>
<li><b><code>sparse_tensor</code>:</b> <font color="black">This is a partially observed tensor which has many missing entries.</font></li>
<li><b><code>time_lags</code>:</b> <font color="black">Time lags, e.g., <code>time_lags = np.array([1, 2, 3])</code>. </font></li>
<li><b><code>alpha</code>:</b> <font color="black">Weights for tensors' nuclear norm, e.g., <code>alpha = np.ones(3) / 3</code>. </font></li>
<li><b><code>rho</code>:</b> <font color="black">Learning rate for ADMM, e.g., <code>rho = 0.0005</code>. </font></li>
<li><b><code>lambda0</code>:</b> <font color="black">Weight for time series regressor, e.g., <code>lambda0 = 5 * rho</code>. If <code>lambda0 = 0</code>, then this imputer is actually a standard low-rank tensor completion (i.e., High-accuracy Low-Rank Tensor Completion, or HaLRTC).</font></li>
<li><b><code>epsilon</code>:</b> <font color="black">Stop criteria, e.g., <code>epsilon = 0.001</code>. </font></li>
<li><b><code>maxiter</code>:</b> <font color="black">Maximum iteration to stop algorithm, e.g., <code>maxiter = 50</code>. </font></li>
</ul>
</div>


In [5]:
def imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho0, lambda0, theta, epsilon, maxiter):
    """Low-Rank Autoregressive Tensor Completion, LATC-imputer."""
    dim = np.array(sparse_tensor.shape)
    dim_time = np.int(np.prod(dim) / dim[0])
    d = len(time_lags)
    max_lag = np.max(time_lags)
    sparse_mat = ten2mat(sparse_tensor, 0)
    pos_missing = np.where(sparse_mat == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    
    X = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{X}}
    T = np.zeros(np.insert(dim, 0, len(dim))) # \boldsymbol{\mathcal{T}}
    Z = sparse_mat.copy()                     # \boldsymbol{Z}
    Z[pos_missing] = np.mean(sparse_mat[sparse_mat != 0])
    A = 0.001 * np.random.rand(dim[0], d)     # \boldsymbol{A}
    it = 0
    ind = np.zeros((d, dim_time - max_lag), dtype = np.int_)
    for i in range(d):
        ind[i, :] = np.arange(max_lag - time_lags[i], dim_time - time_lags[i])
    last_mat = sparse_mat.copy()
    snorm = np.linalg.norm(sparse_mat, 'fro')
    rho = rho0
    while True:
        rho = min(rho * 1.05, 1e5)
        for k in range(len(dim)):
            X[k] = mat2ten(svt_tnn(ten2mat(mat2ten(Z, dim, 0) - T[k] / rho, k), alpha[k] / rho, theta), dim, k)
        tensor_hat = np.einsum('k, kmnt -> mnt', alpha, X)
        mat_hat = ten2mat(tensor_hat, 0)
        mat0 = np.zeros((dim[0], dim_time - max_lag))
        if lambda0 > 0:
            for m in range(dim[0]):
                Qm = mat_hat[m, ind].T
                A[m, :] = np.linalg.pinv(Qm) @ Z[m, max_lag :]
                mat0[m, :] = Qm @ A[m, :]
            mat1 = ten2mat(np.mean(rho * X + T, axis = 0), 0)
            Z[pos_missing] = np.append((mat1[:, : max_lag] / rho), (mat1[:, max_lag :] + lambda0 * mat0) 
                                       / (rho + lambda0), axis = 1)[pos_missing]
        else:
            Z[pos_missing] = (ten2mat(np.mean(X + T / rho, axis = 0), 0))[pos_missing]
        T = T + rho * (X - np.broadcast_to(mat2ten(Z, dim, 0), np.insert(dim, 0, len(dim))))
        tol = np.linalg.norm((mat_hat - last_mat), 'fro') / snorm
        last_mat = mat_hat.copy()
        it += 1
        print('Iter: {}'.format(it))
        print('Tolerance: {:.6}'.format(tol))
        var = dense_tensor[pos_test]
        var_hat = tensor_hat[pos_test]
        print('MAPE: {:.6}'.format(compute_mape(var, var_hat)))
        print('RMSE: {:.6}'.format(compute_rmse(var, var_hat)))
        print()
        if (tol < epsilon) or (it >= maxiter):
            break

    print('Total iteration: {}'.format(it))
    print('Tolerance: {:.6}'.format(tol))
    var = dense_tensor[pos_test]
    var_hat = tensor_hat[pos_test]
    print('Imputation MAPE: {:.6}'.format(compute_mape(var, var_hat)))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(var, var_hat)))
    print()
    
    return tensor_hat

If you want to set parameters reasonably, please use this cross validation on your data set.

### California data - 4W

We generate **random missing (RM)** values on California traffic speed data set.

In [9]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor
gc.collect()

In [10]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 5 * rho
theta = 15
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.652446
MAPE: 0.072237
RMSE: 5.2838

Iter: 2
Tolerance: 0.0630464
MAPE: 0.0295936
RMSE: 2.58715

Iter: 3
Tolerance: 0.0246923
MAPE: 0.0296892
RMSE: 2.47807

Iter: 4
Tolerance: 0.0190302
MAPE: 0.0235354
RMSE: 2.06113

Iter: 5
Tolerance: 0.0171985
MAPE: 0.0199138
RMSE: 1.83126

Iter: 6
Tolerance: 0.0147938
MAPE: 0.0190411
RMSE: 1.77166

Iter: 7
Tolerance: 0.0134758
MAPE: 0.0182681
RMSE: 1.71024

Iter: 8
Tolerance: 0.0123274
MAPE: 0.0178467
RMSE: 1.67521

Iter: 9
Tolerance: 0.0115427
MAPE: 0.0174662
RMSE: 1.65063

Iter: 10
Tolerance: 0.0107518
MAPE: 0.0173537
RMSE: 1.64188

Iter: 11
Tolerance: 0.0102182
MAPE: 0.0170861
RMSE: 1.62474

Iter: 12
Tolerance: 0.00940373
MAPE: 0.0170167
RMSE: 1.61891

Iter: 13
Tolerance: 0.00893174
MAPE: 0.0168292
RMSE: 1.60674

Iter: 14
Tolerance: 0.00838716
MAPE: 0.0167761
RMSE: 1.60238

Iter: 15
Tolerance: 0.00795066
MAPE: 0.016647
RMSE: 1.5936

Iter: 16
Tolerance: 0.00743234
MAPE: 0.0166192
RMSE: 1.591

Iter: 17
Tolerance: 0.00701681
MAPE

In [11]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 4 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor
gc.collect()

In [12]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 5 * rho
theta = 15
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.518
MAPE: 0.100042
RMSE: 7.16416

Iter: 2
Tolerance: 0.0887047
MAPE: 0.062757
RMSE: 4.69289

Iter: 3
Tolerance: 0.0674476
MAPE: 0.0414896
RMSE: 3.42217

Iter: 4
Tolerance: 0.0456977
MAPE: 0.0376394
RMSE: 3.22319

Iter: 5
Tolerance: 0.0274574
MAPE: 0.0383994
RMSE: 3.26981

Iter: 6
Tolerance: 0.0209239
MAPE: 0.0359768
RMSE: 3.10711

Iter: 7
Tolerance: 0.0204584
MAPE: 0.0317411
RMSE: 2.82046

Iter: 8
Tolerance: 0.019987
MAPE: 0.0276616
RMSE: 2.54445

Iter: 9
Tolerance: 0.017436
MAPE: 0.0254248
RMSE: 2.37641

Iter: 10
Tolerance: 0.0148675
MAPE: 0.0246479
RMSE: 2.30032

Iter: 11
Tolerance: 0.0131936
MAPE: 0.024447
RMSE: 2.26884

Iter: 12
Tolerance: 0.0126423
MAPE: 0.0242949
RMSE: 2.25515

Iter: 13
Tolerance: 0.0126387
MAPE: 0.0241002
RMSE: 2.24529

Iter: 14
Tolerance: 0.0127134
MAPE: 0.0238912
RMSE: 2.23964

Iter: 15
Tolerance: 0.0127417
MAPE: 0.0237166
RMSE: 2.23485

Iter: 16
Tolerance: 0.012636
MAPE: 0.0235704
RMSE: 2.23228

Iter: 17
Tolerance: 0.0124476
MAPE: 0.02345

We generate **non-random missing (NM)** values on PeMS traffic speed data set. Then, we conduct the imputation experiment.

In [13]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor
gc.collect()

In [14]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 0.5 * rho
theta = 5
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.651014
MAPE: 0.0999507
RMSE: 7.19214

Iter: 2
Tolerance: 0.0284978
MAPE: 0.0864633
RMSE: 6.38151

Iter: 3
Tolerance: 0.0177231
MAPE: 0.0743862
RMSE: 5.64186

Iter: 4
Tolerance: 0.0147851
MAPE: 0.0652381
RMSE: 5.08395

Iter: 5
Tolerance: 0.0120772
MAPE: 0.0592163
RMSE: 4.7139

Iter: 6
Tolerance: 0.0103575
MAPE: 0.0551239
RMSE: 4.4574

Iter: 7
Tolerance: 0.00895409
MAPE: 0.0523539
RMSE: 4.29003

Iter: 8
Tolerance: 0.00782543
MAPE: 0.0503091
RMSE: 4.17301

Iter: 9
Tolerance: 0.00696545
MAPE: 0.0488652
RMSE: 4.09525

Iter: 10
Tolerance: 0.00636644
MAPE: 0.0477774
RMSE: 4.03942

Iter: 11
Tolerance: 0.00588174
MAPE: 0.0470149
RMSE: 4.00139

Iter: 12
Tolerance: 0.00542859
MAPE: 0.0464299
RMSE: 3.97316

Iter: 13
Tolerance: 0.0050896
MAPE: 0.0460172
RMSE: 3.95325

Iter: 14
Tolerance: 0.00467501
MAPE: 0.0456889
RMSE: 3.93789

Iter: 15
Tolerance: 0.00438012
MAPE: 0.0454529
RMSE: 3.92658

Iter: 16
Tolerance: 0.00406796
MAPE: 0.0452551
RMSE: 3.91752

Iter: 17
Tolerance: 0.00375

In [15]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-4w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 4 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 4 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor
gc.collect()

In [16]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 0.5 * rho
theta = 5
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.51464
MAPE: 0.110016
RMSE: 7.85728

Iter: 2
Tolerance: 0.0406833
MAPE: 0.103609
RMSE: 7.48162

Iter: 3
Tolerance: 0.0246741
MAPE: 0.0967061
RMSE: 7.05266

Iter: 4
Tolerance: 0.0211966
MAPE: 0.0903897
RMSE: 6.66815

Iter: 5
Tolerance: 0.0182744
MAPE: 0.0850268
RMSE: 6.35202

Iter: 6
Tolerance: 0.0165878
MAPE: 0.0802704
RMSE: 6.07302

Iter: 7
Tolerance: 0.0146045
MAPE: 0.076432
RMSE: 5.85444

Iter: 8
Tolerance: 0.0131827
MAPE: 0.0730733
RMSE: 5.66095

Iter: 9
Tolerance: 0.011651
MAPE: 0.0704196
RMSE: 5.5124

Iter: 10
Tolerance: 0.0105019
MAPE: 0.0680932
RMSE: 5.37869

Iter: 11
Tolerance: 0.00929017
MAPE: 0.0662531
RMSE: 5.27586

Iter: 12
Tolerance: 0.00828973
MAPE: 0.0646313
RMSE: 5.18227

Iter: 13
Tolerance: 0.00740335
MAPE: 0.0633258
RMSE: 5.10912

Iter: 14
Tolerance: 0.00662531
MAPE: 0.0621855
RMSE: 5.04363

Iter: 15
Tolerance: 0.00600164
MAPE: 0.0612375
RMSE: 4.99051

Iter: 16
Tolerance: 0.00539077
MAPE: 0.0604212
RMSE: 4.94428

Iter: 17
Tolerance: 0.0049259
MAPE

### California data - 8W

We generate **random missing (RM)** values on California traffic speed data set.

In [17]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.3

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor
gc.collect()

In [18]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 5 * rho
theta = 15
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.652525
MAPE: 0.0771554
RMSE: 5.5723

Iter: 2
Tolerance: 0.0650001
MAPE: 0.0296885
RMSE: 2.59958

Iter: 3
Tolerance: 0.0227401
MAPE: 0.029764
RMSE: 2.46214

Iter: 4
Tolerance: 0.0160032
MAPE: 0.0248274
RMSE: 2.15017

Iter: 5
Tolerance: 0.0134651
MAPE: 0.0207869
RMSE: 1.89864

Iter: 6
Tolerance: 0.0106428
MAPE: 0.0192603
RMSE: 1.77894

Iter: 7
Tolerance: 0.00908942
MAPE: 0.0187583
RMSE: 1.73344

Iter: 8
Tolerance: 0.00827589
MAPE: 0.018378
RMSE: 1.7094

Iter: 9
Tolerance: 0.00758927
MAPE: 0.0181381
RMSE: 1.69616

Iter: 10
Tolerance: 0.00710248
MAPE: 0.0179091
RMSE: 1.68434

Iter: 11
Tolerance: 0.00659525
MAPE: 0.0177718
RMSE: 1.67652

Iter: 12
Tolerance: 0.00620161
MAPE: 0.0176358
RMSE: 1.66884

Iter: 13
Tolerance: 0.00577932
MAPE: 0.0175508
RMSE: 1.66346

Iter: 14
Tolerance: 0.00543289
MAPE: 0.017469
RMSE: 1.65844

Iter: 15
Tolerance: 0.00507347
MAPE: 0.0174173
RMSE: 1.65489

Iter: 16
Tolerance: 0.00478943
MAPE: 0.0173611
RMSE: 1.65131

Iter: 17
Tolerance: 0.0044691

In [6]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_tensor = np.random.rand(data.values.shape[0], 288, 8 * 7)

missing_rate = 0.7

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_tensor, binary_tensor
gc.collect()

In [7]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 5 * rho
theta = 15
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.51719
MAPE: 0.104536
RMSE: 7.38138

Iter: 2
Tolerance: 0.0891542
MAPE: 0.0670867
RMSE: 4.93066

Iter: 3
Tolerance: 0.0679457
MAPE: 0.0434018
RMSE: 3.5385

Iter: 4
Tolerance: 0.0444567
MAPE: 0.0389045
RMSE: 3.31117

Iter: 5
Tolerance: 0.0249864
MAPE: 0.0396496
RMSE: 3.34973

Iter: 6
Tolerance: 0.0177517
MAPE: 0.0372401
RMSE: 3.18929

Iter: 7
Tolerance: 0.0181331
MAPE: 0.0326718
RMSE: 2.88963

Iter: 8
Tolerance: 0.0176761
MAPE: 0.0286207
RMSE: 2.62032

Iter: 9
Tolerance: 0.0149651
MAPE: 0.0265697
RMSE: 2.46377

Iter: 10
Tolerance: 0.0118938
MAPE: 0.0260094
RMSE: 2.3983

Iter: 11
Tolerance: 0.00983847
MAPE: 0.0259276
RMSE: 2.37558

Iter: 12
Tolerance: 0.00917188
MAPE: 0.0258258
RMSE: 2.36581

Iter: 13
Tolerance: 0.00933343
MAPE: 0.0256381
RMSE: 2.35868

Iter: 14
Tolerance: 0.00968258
MAPE: 0.0254337
RMSE: 2.35347

Iter: 15
Tolerance: 0.00986867
MAPE: 0.025256
RMSE: 2.34971

Iter: 16
Tolerance: 0.00983666
MAPE: 0.0251272
RMSE: 2.348

Iter: 17
Tolerance: 0.00965778
MAPE

We generate **non-random missing (NM)** values on California traffic speed data set. Then, we conduct the imputation experiment.

In [6]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.3

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor
gc.collect()

0

In [7]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 0.5 * rho
theta = 5
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 0.649824
MAPE: 0.103234
RMSE: 7.32412

Iter: 2
Tolerance: 0.0287017
MAPE: 0.088324
RMSE: 6.41926

Iter: 3
Tolerance: 0.0169317
MAPE: 0.0744404
RMSE: 5.56306

Iter: 4
Tolerance: 0.0134831
MAPE: 0.064224
RMSE: 4.94375

Iter: 5
Tolerance: 0.0107516
MAPE: 0.0574824
RMSE: 4.53176

Iter: 6
Tolerance: 0.00848518
MAPE: 0.0531022
RMSE: 4.26448

Iter: 7
Tolerance: 0.0068472
MAPE: 0.0501105
RMSE: 4.0915

Iter: 8
Tolerance: 0.00580496
MAPE: 0.0479958
RMSE: 3.97875

Iter: 9
Tolerance: 0.00512817
MAPE: 0.0464893
RMSE: 3.90408

Iter: 10
Tolerance: 0.00458397
MAPE: 0.0454237
RMSE: 3.85501

Iter: 11
Tolerance: 0.00422409
MAPE: 0.0446768
RMSE: 3.82192

Iter: 12
Tolerance: 0.00383202
MAPE: 0.0441579
RMSE: 3.80022

Iter: 13
Tolerance: 0.00361005
MAPE: 0.0437939
RMSE: 3.78501

Iter: 14
Tolerance: 0.00334086
MAPE: 0.0435382
RMSE: 3.77492

Iter: 15
Tolerance: 0.00311149
MAPE: 0.0433536
RMSE: 3.76745

Iter: 16
Tolerance: 0.00289197
MAPE: 0.0432175
RMSE: 3.7623

Iter: 17
Tolerance: 0.0026769

In [8]:
import numpy as np
import pandas as pd
np.random.seed(1000)

data = pd.read_csv('F:/PeMS/California-data-set/pems-8w.csv', header = None)
dense_tensor = mat2ten(data.values, np.array([data.values.shape[0], 288, 8 * 7]), 0)
random_matrix = np.random.rand(data.values.shape[0], 8 * 7)

missing_rate = 0.7

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[2]):
        binary_tensor[i1, :, i2] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)
del data, random_matrix, binary_tensor
gc.collect()

0

In [9]:
import time
start = time.time()
time_lags = np.array([1, 2, 3, 4, 5, 6, 286, 287, 288, 289, 290, 291])
alpha = np.ones(3) / 3
rho = 1e-4
lambda0 = 0.5 * rho
theta = 5
epsilon = 1e-3
maxiter = 100
tensor_hat = imputer(dense_tensor, sparse_tensor, time_lags, alpha, rho, lambda0, theta, epsilon, maxiter)
end = time.time()
print('Running time: %.2f minutes' % ((end - start)/60.0))

Iter: 1
Tolerance: 1.51477
MAPE: 0.115016
RMSE: 8.07865

Iter: 2
Tolerance: 0.0419474
MAPE: 0.10769
RMSE: 7.63958

Iter: 3
Tolerance: 0.0229748
MAPE: 0.0995429
RMSE: 7.12214

Iter: 4
Tolerance: 0.0195844
MAPE: 0.0923165
RMSE: 6.67202

Iter: 5
Tolerance: 0.0170597
MAPE: 0.0860501
RMSE: 6.29137

Iter: 6
Tolerance: 0.0150295
MAPE: 0.0806559
RMSE: 5.96843

Iter: 7
Tolerance: 0.0130102
MAPE: 0.076134
RMSE: 5.70415

Iter: 8
Tolerance: 0.011408
MAPE: 0.0723002
RMSE: 5.4815

Iter: 9
Tolerance: 0.00982494
MAPE: 0.0691576
RMSE: 5.30266

Iter: 10
Tolerance: 0.00868681
MAPE: 0.0665406
RMSE: 5.15365

Iter: 11
Tolerance: 0.00763395
MAPE: 0.0644014
RMSE: 5.03383

Iter: 12
Tolerance: 0.00679444
MAPE: 0.0626156
RMSE: 4.9339

Iter: 13
Tolerance: 0.00602538
MAPE: 0.0611418
RMSE: 4.85311

Iter: 14
Tolerance: 0.00539471
MAPE: 0.0598949
RMSE: 4.78524

Iter: 15
Tolerance: 0.00481986
MAPE: 0.0588546
RMSE: 4.73005

Iter: 16
Tolerance: 0.00433826
MAPE: 0.0579613
RMSE: 4.68315

Iter: 17
Tolerance: 0.00388897
MAP